In [95]:
import sys

sys.path.append('..')

from src.tasks import DelayDiscountTask
from openai import OpenAI

In [96]:
task = DelayDiscountTask(100, ['1 week', '2 weeks', '1 month', '2 months', '6 months', '1 year', '5 years', '25 years'], 6)

In [97]:
client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama'
)

In [98]:
system_message = f"""
You are a participant in a experiment.
You will be given a persona deleminated by the tags <persona> and </persona>.
Act as though you are the person described by the persona.

You will be given a task to complete.
You complete this task by choosing one of the options provided.
Only response with "1" or "2".
No not attempt to provide any other response.

<persona>
You are Mr. Wagner.
</persona>

Task:
"""

In [99]:
class Experiment:
    def __init__(self, task, client, model="llama2", max_retries=3):
        self.task = task
        self.client = client
        self.messages = [{"role": "system", "content":self._get_system_message()}]
        self.model = model
        self.max_retries = max_retries


    def _get_system_message(self):
        system_message = f"You are a participant in a experiment.\nYou will be given a persona deleminated by the tags <persona> and </persona>.\nAct as though you are the person described by the persona.\n\n" + \
        f"You will be given a task to complete.\nYou complete this task by choosing one of the options provided.\nOnly respond with '1' or '2'.\nNo not attempt to provide any other response.\n\n" + \
        f"<persona>\nYou are Mr. Crane.\n</persona>\n\nTask:"

        return system_message


    def _get_choice_message(self):
        choice_1, choice_2 = self.task.get_trial()
        
        return f"{choice_1}\nor\n{choice_2}"
    

    def _get_response(self):
        response = client.chat.completions.create(
        model=self.model,
        messages=self.messages
        )
        
        int_response = -1
        try:
            int_response = int(response.choices[0].message.content)
        except:
            pass

        return int_response
    
    def _retry_response(self):
        self.messages.append({"role": "user", "content": "Respond with 1 or 2 only."})
        response = self._get_response()
        self.messages.append({"role": "assistant", "content": str(response)})
        return response
    
    
    def get_messages(self):
        return self.messages
    

    def run(self):
        while self.task.get_is_running():
            self.messages.append({"role": "user", "content": self._get_choice_message()})
            response = self._get_response()
            self.messages.append({"role": "assistant", "content": str(response)})

            retries = 0
            while response == -1:
                self._retry_response()
                retries += 1
                if retries == self.max_retries:
                    print("Too many retries. Exiting.")
                    return self.task.get_log()

            valid_response = False
            retries = 0
            while not valid_response:
                valid_response = task.record_response(response)
                if not valid_response:
                    self._retry_response()
                    retries += 1
                    if retries == self.max_retries:
                        print("Too many retries. Exiting.")
                        return self.task.get_log()
                    
        
        return self.task.get_log()

In [100]:
test_exp = Experiment(task, client)

In [101]:
log = test_exp.run()

Too many retries. Exiting.


In [102]:
log

[{'trial': 1,
  'delay': '1 week',
  'current_value': 50.0,
  'max_value': 100,
  'max_side': 1,
  'response': 1,
  'chose_max': True}]

In [103]:
test_exp.get_messages()

[{'role': 'system',
  'content': "You are a participant in a experiment.\nYou will be given a persona deleminated by the tags <persona> and </persona>.\nAct as though you are the person described by the persona.\n\nYou will be given a task to complete.\nYou complete this task by choosing one of the options provided.\nOnly respond with '1' or '2'.\nNo not attempt to provide any other response.\n\n<persona>\nYou are Mr. Crane.\n</persona>\n\nTask:"},
 {'role': 'user', 'content': '1: $100.00 in 1 week\nor\n2: $50.00 now'},
 {'role': 'assistant', 'content': '1'},
 {'role': 'user', 'content': '1: $75.00 now\nor\n2: $100.00 in 1 week'},
 {'role': 'assistant', 'content': '-1'},
 {'role': 'user', 'content': 'Respond with 1 or 2 only.'},
 {'role': 'assistant', 'content': '2'},
 {'role': 'user', 'content': 'Respond with 1 or 2 only.'},
 {'role': 'assistant', 'content': '1'},
 {'role': 'user', 'content': 'Respond with 1 or 2 only.'},
 {'role': 'assistant', 'content': '2'}]